In [ ]:
import numpy as np
import time
import psutil
from concurrent.futures import ProcessPoolExecutor
import matplotlib.pyplot as plt


# ----------------------------------------------------
# GENERAR MATRICES
# ----------------------------------------------------
def generate_matrix(n, m):
    return np.random.rand(n, m).astype(np.float64)


# ----------------------------------------------------
# PARALELISMO
# ----------------------------------------------------
def mult_row(A_row, B):
    return np.dot(A_row, B)


def parallel_matrix_multiply(A, B, workers=None):
    n = A.shape[0]
    with ProcessPoolExecutor(max_workers=workers) as executor:
        results = executor.map(mult_row, A, [B]*n)
    return np.vstack(list(results))


# ----------------------------------------------------
# VECTORIZACIÓN
# ----------------------------------------------------
def vectorized_matrix_multiply(A, B):
    return A @ B


# ----------------------------------------------------
# MONITOREO DE RECURSOS
# ----------------------------------------------------
def measure_resource_usage(func, *args, **kwargs):
    """
    Ejecuta la función y mide:
    - Tiempo de ejecución
    - CPU (%) promedio
    - Memoria (MB) pico
    """

    process = psutil.Process()
    cpu_samples = []
    mem_peak = 0

    start_time = time.perf_counter()

    # ----------------------------------------------------
    # Ejecutar mientras medimos recursos
    # ----------------------------------------------------
    result = None

    while True:
        # inicio de ciclo
        cpu_samples.append(psutil.cpu_percent(interval=0.05))

        mem = process.memory_info().rss / (1024**2)
        mem_peak = max(mem_peak, mem)

        if result is None:
            # Ejecutar solo una vez
            result = func(*args, **kwargs)
            end_time = time.perf_counter()
            break

    # ----------------------------------------------------
    # Métricas agregadas
    # ----------------------------------------------------
    time_elapsed = end_time - start_time
    cpu_avg = np.mean(cpu_samples)

    return result, time_elapsed, cpu_avg, mem_peak


# ----------------------------------------------------
# BENCHMARK
# ----------------------------------------------------
def speedup(t_vec, t_par):
    if t_par == 0:
        return float('inf')
    return t_vec / t_par


def efficiency(speedup, threads):
    return speedup / threads


# ----------------------------------------------------
# EXPERIMENTO PRINCIPAL
# ----------------------------------------------------
def run_experiments(sizes, threads):

    results = {
        "size": [],
        "vec_time": [],
        "par_time": [],
        "speedup": [],
        "efficiency": [],

        # Recursos
        "vec_cpu": [],
        "par_cpu": [],
        "vec_mem": [],
        "par_mem": [],
    }

    for n in sizes:
        print(f"\n=== Testing size {n}x{n} ===")

        A = generate_matrix(n, n)
        B = generate_matrix(n, n)

        # ----------------------------------------------------
        # Vectorizado
        # ----------------------------------------------------
        print("Vectorized computing...")
        _, t_vec, cpu_vec, mem_vec = measure_resource_usage(vectorized_matrix_multiply, A, B)
        print(f"Time: {t_vec:.4f} s | CPU avg: {cpu_vec:.1f}% | Mem peak: {mem_vec:.1f} MB")

        # ----------------------------------------------------
        # Paralelo
        # ----------------------------------------------------
        print("Parallel computing...")
        _, t_par, cpu_par, mem_par = measure_resource_usage(parallel_matrix_multiply, A, B, threads)
        print(f"Time: {t_par:.4f} s | CPU avg: {cpu_par:.1f}% | Mem peak: {mem_par:.1f} MB")

        # ----------------------------------------------------
        # Métricas
        # ----------------------------------------------------
        sp = speedup(t_vec, t_par)
        eff = efficiency(sp, threads)

        print(f"Speedup: {sp:.3f} | Efficiency: {eff:.3f}")

        # ----------------------------------------------------
        # Guardar
        # ----------------------------------------------------
        results["size"].append(n)
        results["vec_time"].append(t_vec)
        results["par_time"].append(t_par)
        results["speedup"].append(sp)
        results["efficiency"].append(eff)

        results["vec_cpu"].append(cpu_vec)
        results["par_cpu"].append(cpu_par)
        results["vec_mem"].append(mem_vec)
        results["par_mem"].append(mem_par)

    return results


# ----------------------------------------------------
# PLOTS
# ----------------------------------------------------
def plot_results(results):
    sizes = results["size"]

    # --- tiempo ---
    plt.figure()
    plt.plot(sizes, results["vec_time"], marker="o", label="Vectorized")
    plt.plot(sizes, results["par_time"], marker="o", label="Parallel")
    plt.xlabel("Matrix size (n)")
    plt.ylabel("Time (s)")
    plt.title("Execution Time")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

    # --- speedup ---
    plt.figure()
    plt.plot(sizes, results["speedup"], marker="o")
    plt.xlabel("Matrix size (n)")
    plt.ylabel("Speedup")
    plt.title("Speedup (Vectorized / Parallel)")
    plt.grid()
    plt.tight_layout()
    plt.show()

    # --- eficiencia ---
    plt.figure()
    plt.plot(sizes, results["efficiency"], marker="o")
    plt.xlabel("Matrix size (n)")
    plt.ylabel("Efficiency")
    plt.title("Parallel Efficiency")
    plt.grid()
    plt.tight_layout()
    plt.show()

    # --- CPU ---
    plt.figure()
    plt.plot(sizes, results["vec_cpu"], marker="o", label="Vectorized")
    plt.plot(sizes, results["par_cpu"], marker="o", label="Parallel")
    plt.xlabel("Matrix size (n)")
    plt.ylabel("CPU avg (%)")
    plt.title("CPU Usage")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

    # --- Memoria ---
    plt.figure()
    plt.plot(sizes, results["vec_mem"], marker="o", label="Vectorized")
    plt.plot(sizes, results["par_mem"], marker="o", label="Parallel")
    plt.xlabel("Matrix size (n)")
    plt.ylabel("Peak Memory (MB)")
    plt.title("Memory Usage")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()


# ----------------------------------------------------
# PLOTS LOG-LOG
# ----------------------------------------------------
def plot_loglog(results):
    sizes = results["size"]

    # tiempos loglog
    plt.figure()
    plt.loglog(sizes, results["vec_time"], marker="o", label="Vectorized")
    plt.loglog(sizes, results["par_time"], marker="o", label="Parallel")
    plt.xlabel("Matrix size (log n)")
    plt.ylabel("Time (log s)")
    plt.title("Execution Time (log-log)")
    plt.legend()
    plt.grid()
    plt.tight_layout()
    plt.show()

    # speedup loglog
    plt.figure()
    plt.loglog(sizes, results["speedup"], marker="o")
    plt.xlabel("Matrix size (log n)")
    plt.ylabel("Speedup (log)")
    plt.title("Speedup (log-log)")
    plt.grid()
    plt.tight_layout()
    plt.show()


# ----------------------------------------------------
# MAIN
# ----------------------------------------------------
def main():

    threads = psutil.cpu_count(logical=True)
    print(f"\nDetected threads available: {threads}")

    sizes = [300, 600, 900, 1200]   # puedes cambiarlo

    results = run_experiments(sizes, threads)

    print("\n--- FINAL RESULTS ---")
    for i in range(len(results["size"])):
        print(f"{results['size'][i]}x{results['size'][i]} -> "
              f"vec: {results['vec_time'][i]:.3f}s | "
              f"par: {results['par_time'][i]:.3f}s | "
              f"sp: {results['speedup'][i]:.3f} | "
              f"eff: {results['efficiency'][i]:.3f} | "
              f"vecCPU: {results['vec_cpu'][i]:.1f}% | "
              f"parCPU: {results['par_cpu'][i]:.1f}%")

    plot_results(results)
    plot_loglog(results)


if __name__ == "__main__":
    main()



--- Ejecutando n = 50 ---
basic_mul: 0.0446 s
vectorized (numpy.dot): 0.0001 s
